In [1]:
# Aim of this project is to get the best model for sentiment analysis. I will be trying different methods and seeing the 
# effectiveness of thos methods. The rough outline is that I will be comparing the accuracy of the models using different
# methods like TextBlob and Naive Bayes using different reviews (lemmatized vs non-lemmatized)
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import string
import time
from selenium import webdriver
from sklearn.feature_extraction.text import CountVectorizer


# webscraping
# 1st page : https://en.tripadvisor.com.hk/Restaurant_Review-g294265-d5791814-Reviews-Lau_Pa_Sat-Singapore.html
# 2nd page : https://en.tripadvisor.com.hk/Restaurant_Review-g294265-d5791814-Reviews-or10-Lau_Pa_Sat-Singapore.html

string = "https://en.tripadvisor.com.hk/Restaurant_Review-g294265-d5791814-Reviews"
intermediate = "-or"
end = "-Lau_Pa_Sat-Singapore.html"

comment_list = []

# Want to gain some idea whether it is worth going down to Lau Pa Sat. We will be conducting webscraping using BeautifulSoup.
# But first off, we have to use selenium to automate the pressing of the "more" buttons in the comments before we can use
# BeautifulSoup to scrape the comments. If we don't do this, the reviews scraped will be incomplete.

driver = webdriver.Chrome("C:/webdriver/chromedriver")
driver.get("https://en.tripadvisor.com.hk/Restaurant_Review-g294265-d5791814-Reviews-Lau_Pa_Sat-Singapore.html")

# click the More link (to load all reviews)
driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
time.sleep(5) # Wait for the reviews to load
source = driver.page_source # save the data from a certain page into a variable

In [2]:
# Conduct webscraping on the source 
soup = BeautifulSoup(source, 'lxml')
# print(soup.prettify())

full_comment = ''
for article in soup.find_all('p', class_ = 'partial_entry'):
    comment_list.append(article.text)

In [3]:
# scraping of all the comments will be broken up into 4 parts since I am unsure if the code will break midway
# scraping from pages 2 to 42
for i in range(10,430,10):
    driver = webdriver.Chrome("C:/webdriver/chromedriver")
    driver.get(string + intermediate + str(i) + end)

    # click the More link (to load all reviews)
    driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
    time.sleep(5) 
    source = driver.page_source 
    soup = BeautifulSoup(source, 'lxml')
    # print(soup.prettify())

    full_comment = ''
    for article in soup.find_all('p', class_ = 'partial_entry'):
        comment_list.append(article.text)
# len(comment_list) is 430

In [4]:
comment_list2 = []
# scraping from pages 43 to 86
for i in range(430,870,10):
    driver = webdriver.Chrome("C:/webdriver/chromedriver")
    driver.get(string + intermediate + str(i) + end)

    # click the More link (to load all reviews)
    button = driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']")
    driver.execute_script("arguments[0].click();", button)
    
    time.sleep(5) 
    source = driver.page_source 
    soup = BeautifulSoup(source, 'lxml')
    # print(soup.prettify())

    full_comment = ''
    for article in soup.find_all('p', class_ = 'partial_entry'):
        comment_list2.append(article.text)
# len(comment_list2) is 440

In [5]:
comment_list3 = []
# scraping from pages 87 to 128
for i in range(870,1290,10):
    driver = webdriver.Chrome("C:/webdriver/chromedriver")
    driver.get(string + intermediate + str(i) + end)

    # click the More link (to load all reviews)
    button = driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(5) 
    source = driver.page_source 
    soup = BeautifulSoup(source, 'lxml')
    # print(soup.prettify())

    full_comment = ''
    for article in soup.find_all('p', class_ = 'partial_entry'):
        comment_list3.append(article.text)
    
# len(comment_list3) = 420

In [6]:
comment_list4 = []
# scraping from pages 129 to 168 (final page)
for i in range(1290,1680,10):
    driver = webdriver.Chrome("C:/webdriver/chromedriver")
    driver.get(string + intermediate + str(i) + end)

    # click the More link (to load all reviews)
    button = driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(5) 
    source = driver.page_source 
    soup = BeautifulSoup(source, 'lxml')
    # print(soup.prettify())

    full_comment = ''
    for article in soup.find_all('p', class_ = 'partial_entry'):
        comment_list4.append(article.text)
# have to combine all the 4 comment_lists into one at the end
# len(comment_list4) = 390

In [7]:
full_comments = comment_list + comment_list2 + comment_list3 + comment_list4
rev_num_list = []
string2 = "Review"
for i in range(1,1681):
    rev_num_list.append(string2 + str(i))

In [8]:
# up next want to get the ratings for all the reviews. I will use the bubble ratings in the website as the target concept
# page 1
source = requests.get("https://en.tripadvisor.com.hk/Restaurant_Review-g294265-d5791814-Reviews-Lau_Pa_Sat-Singapore.html").text
soup = BeautifulSoup(source, 'lxml')
bubblerating = []
for article in soup.find_all('div',class_ = "ui_column is-9"):
    if pd.isnull(article.find('span', class_ = re.compile("ui_bubble_rating bubble\d*"))):
        continue
    else:
        bubblerating.append(article.find('span', class_ = re.compile("ui_bubble_rating bubble\d*")))

In [9]:
# get bubble ratings for the rest of the pages (pages 2 - 168). Bubble ratings are scores upon 5 denoting how a person felt  
for i in range(10,1680,10):
    source = requests.get(string + intermediate + str(i) + end).text
    soup = BeautifulSoup(source, 'lxml')
    for article in soup.find_all('div',class_ = "ui_column is-9"):
        if pd.isnull(article.find('span', class_ = re.compile("ui_bubble_rating bubble\d*"))):
            continue
        else:
            bubblerating.append(article.find('span', class_ = re.compile("ui_bubble_rating bubble\d*")))

In [10]:
ratings = re.findall('\d+', ''.join(map(str, bubblerating)))
mod_ratings = []
for i in ratings:
    mod_ratings.append(int(i)/10)
# mod_ratings

[5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 3.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 4.0,
 5.0,
 3.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 4.0,
 4.0,
 3.0,
 4.0,
 5.0,
 1.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 4.0,
 4.0,
 3.0,
 4.0,
 3.0,
 4.0,
 3.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 3.0,
 4.0,
 4.0,
 5.0,
 4.0,
 1.0,
 5.0,
 3.0,
 4.0,
 4.0,
 1.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 1.0,
 3.0,
 4.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 4.0,
 5.0,
 1.0,
 1.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 5.0,
 4.0,
 4.0,
 3.0,
 4.0,
 1.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 4.0,
 5.0,
 5.0,
 5.0,
 3.0,
 4.0,
 5.0,
 5.0,
 2.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 4.0,
 5.0,
 3.0,
 4.0,
 5.0,
 4.0,
 1.0,
 2.0,
 3.0,
 3.0,
 4.0,
 4.0,
 5.0,
 3.0,
 4.0,
 5.0,
 3.0,
 5.0,
 3.0,
 5.0,
 4.0,
 5.0,
 1.0,
 5.0,
 3.0,
 1.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 4.0

In [11]:
# Form the corpus
df = pd.DataFrame(list(zip(rev_num_list,full_comments,mod_ratings)), columns = ['review_num','review','rating']) 
df

,review_num,review,rating
0,Review1,Always love to visit here. There are so many o...,5.0
1,Review2,A nice tourist attraction to visit and enjoy l...,4.0
2,Review3,Gd place for breakfast especially those who ar...,4.0
3,Review4,Great fun and interesting food some not to my ...,4.0
4,Review5,"10/10, sooo good!! Best butter chicken and naa...",5.0
...,...,...,...
1674,Review1675,We really like this old hawker center. Unfort...,3.0
1675,Review1676,It was still under renovation during my visit ...,3.0
1676,Review1677,Went to Best Satay Number 7 on Boon Tat St. Ex...,5.0
1677,Review1678,Lau Pa Sat is the place for satay! At promptly...,5.0


In [12]:
import string 
# Before tokenizing the sentences, I will carry out some cleaning.
def remove_punctuations(text):
    for puncutation in string.punctuation:
        text = text.replace(puncutation,'')
    return text
def clean_rd_1(sentence):
    '''Make the sentences into lowercase, remove punctuations and remove words that contain numbers'''
    sentence = sentence.lower()
    sentence = sentence.replace("wasn't","was not")
    sentence = sentence.replace("it's",'it is')
    sentence = sentence.replace("you're",'you are')
    sentence = sentence.replace("sooo",'so')
    sentence = sentence.replace("lau pa sat",'')
    sentence = re.sub("^gd", "good", sentence)
    sentence = re.sub('\[.*?\/]', '', sentence)
    sentence = re.sub('\w*\d\w*','',sentence)
    sentence = re.sub('\n','',sentence)
    return sentence
                      
clean = lambda x: clean_rd_1(x)
remove_punc = lambda x: remove_punctuations(x)
df.review = df['review'].apply(clean)
df.review = df['review'].apply(remove_punc)
df

,review_num,review,rating
0,Review1,always love to visit here there are so many op...,5.0
1,Review2,a nice tourist attraction to visit and enjoy l...,4.0
2,Review3,good place for breakfast especially those who ...,4.0
3,Review4,great fun and interesting food some not to my ...,4.0
4,Review5,so good best butter chicken and naan amazing...,5.0
...,...,...,...
1674,Review1675,we really like this old hawker center unfortu...,3.0
1675,Review1676,it was still under renovation during my visit ...,3.0
1676,Review1677,went to best satay number on boon tat st exce...,5.0
1677,Review1678,is the place for satay at promptly pm the st...,5.0


In [13]:
# We will form bag of words using the corpus for data exploration later on

cv = CountVectorizer(stop_words = 'english')
data_cv = cv.fit_transform(df['review'])
data_dtm = pd.DataFrame(data_cv.toarray(), columns = cv.get_feature_names())
data_dtm
# from countvectorizer, we can see that chinese words are present. As such, we will remove the row containing chinese
# df[df['review'].str.contains("这里的店员都十分热情")]
# we can see that the 1508th row of the dataframe contains chinese. It will be removed 

,aaa,abdomen,ability,able,abound,aboutthen,abrubt,absolulety,absolute,absolutely,...,zone,zones,交通十分便利,位于cbd地带,十分惬意,我们是在这里享受了下午茶,服务也比其他local,然后再慢步marina,而且,这里的店员都十分热情
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1676,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1677,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

# df2 will be used to lemmatize the words in the reviews 
df2 = df.copy()
tokenize = lambda x: word_tokenize(x)
df2['review'] = df2['review'].apply(tokenize)

# conducting pos_tag so that I can lemmatize the words 
def tagging(lis):
    tag_list = []
    for i in lis:
        tag_list.append(nltk.pos_tag([i]))
    return tag_list
tag = lambda x: tagging(x)
df2['review'] = df2['review'].apply(tag)
df2

,review_num,review,rating
0,Review1,"[[(always, RB)], [(love, NN)], [(to, TO)], [(v...",5.0
1,Review2,"[[(a, DT)], [(nice, JJ)], [(tourist, NN)], [(a...",4.0
2,Review3,"[[(good, JJ)], [(place, NN)], [(for, IN)], [(b...",4.0
3,Review4,"[[(great, JJ)], [(fun, NN)], [(and, CC)], [(in...",4.0
4,Review5,"[[(so, RB)], [(good, JJ)], [(best, JJS)], [(bu...",5.0
...,...,...,...
1674,Review1675,"[[(we, PRP)], [(really, RB)], [(like, IN)], [(...",3.0
1675,Review1676,"[[(it, PRP)], [(was, VBD)], [(still, RB)], [(u...",3.0
1676,Review1677,"[[(went, VBD)], [(to, TO)], [(best, JJS)], [(s...",5.0
1677,Review1678,"[[(is, VBZ)], [(the, DT)], [(place, NN)], [(fo...",5.0


In [15]:
def wordnet_tags(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
def change_tag(x):
    lis = []
    for i in x:
        lis.append([i[0][0], wordnet_tags(i[0][1])])
    return lis
df2['changed_tag'] = df2['review'].apply(change_tag)
df2
# change_tag(df2.iloc[0,1])

,review_num,review,rating,changed_tag
0,Review1,"[[(always, RB)], [(love, NN)], [(to, TO)], [(v...",5.0,"[[always, r], [love, n], [to, n], [visit, n], ..."
1,Review2,"[[(a, DT)], [(nice, JJ)], [(tourist, NN)], [(a...",4.0,"[[a, n], [nice, a], [tourist, n], [attraction,..."
2,Review3,"[[(good, JJ)], [(place, NN)], [(for, IN)], [(b...",4.0,"[[good, a], [place, n], [for, n], [breakfast, ..."
3,Review4,"[[(great, JJ)], [(fun, NN)], [(and, CC)], [(in...",4.0,"[[great, a], [fun, n], [and, n], [interesting,..."
4,Review5,"[[(so, RB)], [(good, JJ)], [(best, JJS)], [(bu...",5.0,"[[so, r], [good, a], [best, a], [butter, n], [..."
...,...,...,...,...
1674,Review1675,"[[(we, PRP)], [(really, RB)], [(like, IN)], [(...",3.0,"[[we, n], [really, r], [like, n], [this, n], [..."
1675,Review1676,"[[(it, PRP)], [(was, VBD)], [(still, RB)], [(u...",3.0,"[[it, n], [was, v], [still, r], [under, n], [r..."
1676,Review1677,"[[(went, VBD)], [(to, TO)], [(best, JJS)], [(s...",5.0,"[[went, v], [to, n], [best, a], [satay, n], [n..."
1677,Review1678,"[[(is, VBZ)], [(the, DT)], [(place, NN)], [(fo...",5.0,"[[is, v], [the, n], [place, n], [for, n], [sat..."


In [16]:
# lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatizing(x):
    lis = []
    for i in x:
        lis.append(lemmatizer.lemmatize(i[0],i[1]))
    return lis
df2['lemmatized'] = df2['changed_tag'].apply(lemmatizing)
df2

,review_num,review,rating,changed_tag,lemmatized
0,Review1,"[[(always, RB)], [(love, NN)], [(to, TO)], [(v...",5.0,"[[always, r], [love, n], [to, n], [visit, n], ...","[always, love, to, visit, here, there, be, so,..."
1,Review2,"[[(a, DT)], [(nice, JJ)], [(tourist, NN)], [(a...",4.0,"[[a, n], [nice, a], [tourist, n], [attraction,...","[a, nice, tourist, attraction, to, visit, and,..."
2,Review3,"[[(good, JJ)], [(place, NN)], [(for, IN)], [(b...",4.0,"[[good, a], [place, n], [for, n], [breakfast, ...","[good, place, for, breakfast, especially, thos..."
3,Review4,"[[(great, JJ)], [(fun, NN)], [(and, CC)], [(in...",4.0,"[[great, a], [fun, n], [and, n], [interesting,...","[great, fun, and, interest, food, some, not, t..."
4,Review5,"[[(so, RB)], [(good, JJ)], [(best, JJS)], [(bu...",5.0,"[[so, r], [good, a], [best, a], [butter, n], [...","[so, good, best, butter, chicken, and, naan, a..."
...,...,...,...,...,...
1674,Review1675,"[[(we, PRP)], [(really, RB)], [(like, IN)], [(...",3.0,"[[we, n], [really, r], [like, n], [this, n], [...","[we, really, like, this, old, hawker, center, ..."
1675,Review1676,"[[(it, PRP)], [(was, VBD)], [(still, RB)], [(u...",3.0,"[[it, n], [was, v], [still, r], [under, n], [r...","[it, be, still, under, renovation, during, my,..."
1676,Review1677,"[[(went, VBD)], [(to, TO)], [(best, JJS)], [(s...",5.0,"[[went, v], [to, n], [best, a], [satay, n], [n...","[go, to, best, satay, number, on, boon, tat, s..."
1677,Review1678,"[[(is, VBZ)], [(the, DT)], [(place, NN)], [(fo...",5.0,"[[is, v], [the, n], [place, n], [for, n], [sat...","[be, the, place, for, satay, at, promptly, pm,..."


In [17]:
df2['review'] = df2['lemmatized']
df2 = df2.drop(['changed_tag', 'lemmatized'], axis = 1)
df2

,review_num,review,rating
0,Review1,"[always, love, to, visit, here, there, be, so,...",5.0
1,Review2,"[a, nice, tourist, attraction, to, visit, and,...",4.0
2,Review3,"[good, place, for, breakfast, especially, thos...",4.0
3,Review4,"[great, fun, and, interest, food, some, not, t...",4.0
4,Review5,"[so, good, best, butter, chicken, and, naan, a...",5.0
...,...,...,...
1674,Review1675,"[we, really, like, this, old, hawker, center, ...",3.0
1675,Review1676,"[it, be, still, under, renovation, during, my,...",3.0
1676,Review1677,"[go, to, best, satay, number, on, boon, tat, s...",5.0
1677,Review1678,"[be, the, place, for, satay, at, promptly, pm,...",5.0


In [18]:
def joining(x):
     return ' '.join(x)
df2 = df2.copy()
df2['review'] = df2['review'].apply(joining)
df2

,review_num,review,rating
0,Review1,always love to visit here there be so many opt...,5.0
1,Review2,a nice tourist attraction to visit and enjoy l...,4.0
2,Review3,good place for breakfast especially those who ...,4.0
3,Review4,great fun and interest food some not to my tas...,4.0
4,Review5,so good best butter chicken and naan amazingly...,5.0
...,...,...,...
1674,Review1675,we really like this old hawker center unfortun...,3.0
1675,Review1676,it be still under renovation during my visit i...,3.0
1676,Review1677,go to best satay number on boon tat st excelle...,5.0
1677,Review1678,be the place for satay at promptly pm the stre...,5.0


In [19]:
import pickle
# saving df and df2 using pickle for backup if I make a careless mistake
df.to_pickle("initial_df.pkl")
df2.to_pickle("initial_df2.pkl")

In [24]:
# I will be dropping the 1508th row for both df and df2
df = df.drop(1508, axis = 0)
df2 = df2.drop(1508, axis = 0)
df2

,review_num,review,rating
0,Review1,always love to visit here there be so many opt...,5.0
1,Review2,a nice tourist attraction to visit and enjoy l...,4.0
2,Review3,good place for breakfast especially those who ...,4.0
3,Review4,great fun and interest food some not to my tas...,4.0
4,Review5,so good best butter chicken and naan amazingly...,5.0
...,...,...,...
1674,Review1675,we really like this old hawker center unfortun...,3.0
1675,Review1676,it be still under renovation during my visit i...,3.0
1676,Review1677,go to best satay number on boon tat st excelle...,5.0
1677,Review1678,be the place for satay at promptly pm the stre...,5.0


In [25]:
# Bag of words is produced for use in topic modelling 
from sklearn.feature_extraction.text import CountVectorizer

cv2 = CountVectorizer(stop_words = 'english')
data_cv2 = cv2.fit_transform(df2.review)
data_dtm2 = pd.DataFrame(data_cv2.toarray(), columns = cv2.get_feature_names())
data_dtm2.index = df2.index
data_dtm2

,aaa,abdomen,ability,able,abound,aboutthen,abrubt,absolulety,absolute,absolutely,...,yu,yum,yummm,yummy,zao,zealous,zha,zingy,zion,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1676,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1677,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
import pickle
df.to_pickle("df.pkl")
df2.to_pickle("df2.pkl")
# data_dtm refers to the document-term matrix produced using df
# data_dtm2 refers to the document-term matrix produced using df2
data_dtm.to_pickle("data_dtm.pkl")
data_dtm2.to_pickle("data_dtm2.pkl")
# Note that I am not done with the pre-processing. I will be continuing with 
# it in round 2 of data cleaning ipynb. Reason is that if I accidentally mess up, I will not need
# to re-run all the code chunks in this ipynb, which is really time consuming.